In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [2]:
from util import *
import numpy as np
from numpy.linalg import inv, matrix_rank
import json

In [3]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix_Sioux.pkz')
P = np.matrix(P)

print('rank of P is: ')
print(matrix_rank(P))

print('shape of P is: ')
print(np.shape(P))

rank of P is: 
552
shape of P is: 
(552, 14232)


In [4]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix_Sioux-Falls.pkz')

print('rank of A is: ')
print(matrix_rank(A))

print('shape of A is: ')
print(np.shape(A))

rank of A is: 
76
shape of A is: 
(76, 14232)


In [5]:
# load link counts data

flow_list = []
with open('SiouxFallsFlow.txt', 'r') as f:
    read_data = f.readlines()
    flag = 0
    for row in read_data:
        flag += 1
        if flag > 1:
            flow_list.append(float(row.split('\t')[2]))

x = np.array(flow_list)

In [6]:
x

array([  4494.65764646,   8119.07994805,   4519.07994805,   5967.33639617,
         8094.65764646,  14006.37101986,  10022.31961516,  14030.5609174 ,
        18006.37101986,   5200.        ,  18030.5609174 ,   8798.26771411,
        15780.78205547,   5991.75869776,   8806.49866681,  12492.92536056,
        12101.52912231,  15794.01060698,  12525.57861486,  12040.91827285,
         6882.66491266,   8388.713063  ,  15796.7410003 ,   6836.70597529,
        21744.07608018,  21814.07608764,  17726.62503296,  23125.7972901 ,
        11047.09388127,   8100.        ,   5300.        ,  17604.22353323,
         8365.28565386,   9776.11953275,   9973.70741603,   8404.93462395,
        12287.60526902,  12378.64203998,  11121.35796002,   9814.06906293,
         9036.33413403,   8400.43683027,  23192.28335936,   9079.82031659,
        19083.28976475,  18409.93502652,   8406.71440521,  11073.00931921,
        11695.00291653,  15278.32524152,   8100.        ,  11683.83828244,
         9953.02143205,  

### Assignment Equation

We have the following equation: 
$$AP'\boldsymbol{\lambda} = \textbf{x},$$
whose least-squares solution can be written as
$$\boldsymbol{\lambda} = (AP')^+\textbf{x}, \quad (1)$$
where $(AP')^{+}$ is the pseudo-inverse of $AP'$.

However, the $\boldsymbol{\lambda}$ given by (1) might contain negative entries, which is not desired. Thus, instead, we solve a constrained least-squares problem:
$$\mathop {\min }\limits_{\boldsymbol{\lambda}  \geq \textbf{0}} {\left\| {AP'\boldsymbol{\lambda}  - \textbf{x}} \right\|_2}. \quad (2)$$

Note that (2) typically contains a non-PSD matrix Q, thus preventing the solver calculating the correct $\boldsymbol{\lambda}$.

In the end, we give up solving (2), but adjust the solution given by (1) such that all entries in $\boldsymbol{\lambda}$ are positive or zero.

In [7]:
P_t = np.transpose(P)
# AP'
AP_t = np.dot(A, P_t)

pinv_AP_t = LA.pinv(AP_t)

lam_0 = np.dot(pinv_AP_t, x)

In [8]:
# assert(1==2)

L = 24 * (24 - 1)  # dimension of lam

model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective
obj = 0

for l in range(L):
    obj += (lam[l] - lam_0[0, l]) * (lam[l] - lam_0[0, l])
    
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

# fictitious_OD_list = zload('fictitious_OD_list')
# for l in fictitious_OD_list:
#     model.addConstr(lam[l] == 0)
model.update() 

model.setParam('OutputFlag', False)
model.optimize()

lam_list = []
for v in model.getVars():
    # print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
# print('Obj: %g' % obj.getValue())

In [9]:
# write estimation result to file
n = 24  # number of nodes
with open('OD_demand_matrix_Sioux.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1